In [144]:
import casadi as ca
import numpy as np

In [145]:
#
dt = 0.01
N = 10;
# gravity
_gz = 9.81
# thrust and angular rates limits
_w_max_yaw = 6.0
_w_max_xy = 6.0
_thrust_min = 2.0
_thrust_max = 20.0
_s_dim = 10
_u_dim = 4

In [146]:
#Cost matrix for tracking the goal point
Q = np.diag([
            100, 100, 100,  # delta_x, delta_y, delta_z
            10, 10, 10, 10, # delta_qw, delta_qx, delta_qy, delta_qz
            10, 10, 10])

# cost matrix for the action
R = np.diag([0.1, 0.1, 0.1, 0.1]) # T, wx, wy, wz

In [147]:
#states
px, py, pz = ca.MX.sym('px'), ca.MX.sym('py'), ca.MX.sym('pz')
        #
qw, qx, qy, qz = ca.MX.sym('qw'), ca.MX.sym('qx'), ca.MX.sym('qy'), \
    ca.MX.sym('qz')
#
vx, vy, vz = ca.MX.sym('vx'), ca.MX.sym('vy'), ca.MX.sym('vz')

# -- conctenated vector
_x = ca.vertcat(px, py, pz, qw, qx, qy, qz, vx, vy, vz) 

In [148]:
#control command
thrust, wx, wy, wz = ca.MX.sym('thrust'), ca.MX.sym('wx'), \
            ca.MX.sym('wy'), ca.MX.sym('wz')
_u = ca.vertcat(thrust, wx, wy, wz)

In [149]:
# system dynamics
x_dot = ca.vertcat(
            vx,
            vy,
            vz,
            0.5 * ( -wx*qx - wy*qy - wz*qz ),
            0.5 * (  wx*qw + wz*qy - wy*qz ),
            0.5 * (  wy*qw - wz*qx + wx*qz ),
            0.5 * (  wz*qw + wy*qx - wx*qy ),
            2 * ( qw*qy + qx*qz ) * thrust,
            2 * ( qy*qz - qw*qx ) * thrust, 
            (qw*qw - qx*qx -qy*qy + qz*qz) * thrust - _gz
        )

        #
f = ca.Function('f', [_x, _u], [x_dot], ['x', 'u'], ['ode'])

In [150]:
f

Function(f:(x[10],u[4])->(ode[10]) MXFunction)

In [151]:
# RK4 integration
k1 = f(_x, _u)
k2 = f(_x + dt/2*k1, _u)
k3 = f(_x + dt/2*k2, _u)
k4 = f(_x + dt * k3, _u)
x_next = _x + (dt / 6) * (k1 + 2 * k2 + 2 * k3 + k4)
F = ca.Function('F', [_x, _u], [x_next], ['x', 'u'], ['F'])

In [152]:
F

Function(F:(x[10],u[4])->(F[10]) MXFunction)

In [153]:
sim = F.mapaccum(N)
sim

Function(F_acc10:(x[10],u[4x10])->(F[10x10]) MXFunction)

In [154]:
opti = ca.Opti()
X = opti.variable(_s_dim,N+1)
U = opti.variable(_u_dim, N)
P = opti.parameter(_s_dim * 2) # contains the current state and target state

In [155]:
u_min = [_thrust_min, -_w_max_xy, -_w_max_xy, -_w_max_yaw]
u_max = [_thrust_max,  _w_max_xy,  _w_max_xy,  _w_max_yaw]

In [156]:
u_min

[2.0, -6.0, -6.0, -6.0]

In [157]:
mpc_obj = 0
for k in range(N):
    st = X[:, k]
    st_next = X[:, k+1]
    con = U[:, k] - [_gz , 0, 0, 0]
    mpc_obj += (st - P[_s_dim:]).T @ Q @ (st - P[_s_dim:]) \
                    + con.T @ R @ con
    st_next_RK4 = F(st, con)
    opti.subject_to(st_next_RK4==st_next)
    opti.subject_to(con>= u_min)
    opti.subject_to(con<=u_max)
    
opti.subject_to(X[:,0] == P[:_s_dim])
opti.minimize(mpc_obj)

In [158]:
opti

Opti(Opti {
  instance #4
  #variables: 2 (nx = 150)
  #parameters: 1 (np = 20)
  #constraints: 31 (ng = 190)
  CasADi solver needs updating.
})

In [159]:
opts = {
            'ipopt': {
                'max_iter': 2000,
                'print_level': 0,
                'acceptable_tol': 1e-8,
                'acceptable_obj_change_tol': 1e-6
            },
            'print_time': 0
        }

opti.solver('ipopt', opts)

In [160]:
state_current = np.array([0, 0 , 0, 0, 0, 0, 0, 0, 0, 0])
state_target = np.array([0, 0 , 0, 0, 0, 0, 0, 0, 0, 0])
param = np.concatenate((state_current, state_target))


In [161]:
opti.set_value(P, param)


In [162]:
sol = opti.solve()

In [163]:
u_opt = sol.value(U)

In [164]:
u_opt[:,0]

array([11.80999999,  0.        ,  0.        ,  0.        ])

In [165]:
M =opti.to_function('mpc', [P],[U[:,0]], ['p'], ['u_opt'] )

In [166]:
M.save('NMPC.casadi')
M2 = ca.Function.load('NMPC.casadi')

In [167]:

state_current = np.array([3.91536618, -0.02897329 , 1.37337641, 1, 0, 0, 0, 0, 0, 0])
state_target = np.array([3.91536618, -0.02897329 , 1.37337641, 1, 0, 0, 0, 0, 0, 0])
param = np.concatenate((state_current, state_target))

In [168]:
opt_u = M2(param)

In [169]:
opt_u.toarray()[0,0]

13.036033161541289